In [1]:
import sys
sys.path.append('..')
from config_notebooks import *
from qdrant.qdrant_helper import *
from azureopenaimanager.azureopenai_helper import AzureOpenAIManager
from azure_blob.azure_blob_helper import AzureBlobHelper
from sentence_transformers import CrossEncoder

c:\Ambarish\text2sql\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
search = QdrantHelper(QDRANT_URL,QDRANT_KEY,MODEL_NAME,QDRANT_COLLECTION,
                      qmodels.Distance.COSINE, 384,25)

In [10]:
query = """
stomata 
"""

In [11]:
CATEGORY = "NCERT-CLASS11"
USER_ID = "y12ft5bl30ue"

In [12]:
results_to_return, \
metadata_source_filename_to_return, \
metadata_source_page_to_return, \
reranker_score_to_return = search.get_search_results(user_input=query,
                                                     CATEGORY=CATEGORY,
                                                     user_id=USER_ID,
                                                     )


stomata 

################################
[ 5.00665233e-02  6.25008866e-02 -1.57310843e-01  2.81142853e-02
 -9.93715152e-02 -1.07159205e-02  8.31121877e-02  7.72103807e-03
  4.20955420e-02 -4.62356061e-02 -7.03867711e-03 -5.84046170e-03
 -4.14371490e-02 -5.00573888e-02 -3.52696925e-02 -1.56877711e-02
  2.61332002e-02  3.93955633e-02  3.12556103e-02 -3.63989994e-02
 -7.33106583e-03  4.73934179e-03 -5.53059019e-03  4.46108133e-02
  7.96023197e-03  1.12505928e-01 -3.87314409e-02  2.28370950e-02
 -3.09674088e-02 -1.14927880e-01 -1.68372039e-02  1.35696474e-02
  7.46266842e-02 -1.63810644e-02 -3.66131105e-02  7.72001594e-02
 -6.96768388e-02 -8.78195688e-02 -2.16445327e-02  9.77210552e-02
 -3.68086286e-02 -2.70062052e-02 -9.76544246e-03  4.92251813e-02
 -1.32038698e-01 -8.51807967e-02  1.25046391e-02  6.79176822e-02
  1.24001643e-02 -1.87688414e-02 -4.89797890e-02 -1.30609751e-01
 -8.59092474e-02 -4.63484973e-03 -2.91612055e-02  5.25934040e-04
 -6.07022569e-02  2.72390395e-02  8.92100018e-

In [13]:
metadata_source_filename_to_return

['CHAP06-BIOLOGY-CLASS11.pdf']

In [14]:
metadata_source_page_to_return

['CHAP06-BIOLOGY-CLASS11-1.pdf']

In [15]:
reranker_score_to_return

[]

In [16]:
results_to_return[0]

' The outside of the\nepidermis is often covered with a waxy thick layer called the cuticle  which\nprevents the loss of water . Cuticle is absent in r oots. Stomata  are structur es\npresent in the epidermis of leaves. Stomata regulate the process of\ntranspiration and gaseous exchange. Each stoma is composed of two bean-\nshaped cells known as guard cells  which enclose stomatal pore. In\ngrasses, the guard cells are dumb-bell shaped. The outer walls of guard\ncells (away from the stomatal pore) are thin and the inner walls (towards\nthe stomatal pore) are highly thickened. The guard cells possess chloroplasts\nand regulate the opening and closing of stomata. Sometimes, a few epidermal\ncells, in the vicinity of the guard cells become specialised in their shape and\nsize and are known as subsidiary cells . The stomatal aperture, guard\ncells and the surrounding subsidiary cells are together called stomatal\napparatus (Figure 6. 1).\nFigure 6.1 Diagrammatic representation: (a) stomata

In [17]:
#We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

c:\Ambarish\text2sql\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ambarish Ganguly\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [19]:
for i in range(len(results_to_return)):
               print(results_to_return[i])

 The outside of the
epidermis is often covered with a waxy thick layer called the cuticle  which
prevents the loss of water . Cuticle is absent in r oots. Stomata  are structur es
present in the epidermis of leaves. Stomata regulate the process of
transpiration and gaseous exchange. Each stoma is composed of two bean-
shaped cells known as guard cells  which enclose stomatal pore. In
grasses, the guard cells are dumb-bell shaped. The outer walls of guard
cells (away from the stomatal pore) are thin and the inner walls (towards
the stomatal pore) are highly thickened. The guard cells possess chloroplasts
and regulate the opening and closing of stomata. Sometimes, a few epidermal
cells, in the vicinity of the guard cells become specialised in their shape and
size and are known as subsidiary cells . The stomatal aperture, guard
cells and the surrounding subsidiary cells are together called stomatal
apparatus (Figure 6. 1).
Figure 6.1 Diagrammatic representation: (a) stomata with bean-shap

In [20]:
# Now, score all retrieved passages with the cross_encoder
cross_inp = [[query, results_to_return[i]] for i in range(len(results_to_return))]
cross_scores = cross_encoder.predict(cross_inp)

In [21]:
cross_scores

array([6.3924537], dtype=float32)

In [ ]:
def get_reply(user_input, content):

    azure_open_ai_manager = AzureOpenAIManager(
                    endpoint=AZURE_OPENAI_ENDPOINT,
                    api_key=AZURE_OPENAI_KEY,
                    deployment_id=AZURE_OPENAI_DEPLOYMENT_ID,
                    api_version="2023-05-15"
                )
    
    conversation=[{"role": "system", "content": "If the answer is not found within the context, please mention \
        that the answer is not found \
        Do not answer anything which is not in the context"}]
    reply = azure_open_ai_manager.generate_reply_from_context(user_input, content, conversation)
    return reply

In [ ]:
azure_blob_helper = AzureBlobHelper(AZ_ST_ACC_NAME,
                                    AZ_ST_ACC_KEY,
                                    AZ_ST_CONTAINER_NAME)

In [ ]:
context = "\n".join(results_to_return)
reply = get_reply(query, context)

URLs = []

reranker_confidence = ""

for page in metadata_source_page_to_return:
    URLs.append(azure_blob_helper.generate_sas_url(page))


In [ ]:
reply